This is the bayesian optimizer with an example

In [1]:
#imports
import numpy as np
from bayes_opt import BayesianOptimization
from scipy import integrate

In [2]:
#define the times measured over
timespan = [0, 1, 3, 5, 7, 9, 11, 13, 15, 19, 23, 27, 31, 40, 50, 60, 90, 120, 150, 180, 240, 300, 360, 440, 530, 620, 810, 900, 955, 1246]

In [3]:
#function that runs the guesses of the reaction condition optimization algorithm with parameters found by chemfit
def get_experiment(T,P,S0,Ccat0): #the function takes the to optimize variables
    'This function is to illustrate the bayesian optimizer'
    R= 8.314 #define needed constants
    KH= 1282
    CH2 = P/KH
    k1 = 3.33*np.exp(-17041/(R*T)) #now if i have found my arrhenius constants and activation energies i put them in like this
    k2f = 1.97*np.exp(-14699/(R*T))
    k2b = 1.259*np.exp(-17180/(R*T))
    k3 = 4.09*np.exp(-11794/(R*T))
    k4 = 1.92*np.exp(-12290/(R*T))
    k5 = 32.9*np.exp(-23902/(R*T))
    init = [S0,0,0,0,Ccat0,0,0,0] #define how the initial conditon is made from the optimized variables
    def diff(x, init): #the differential system to simulate
        CE, CA1, CA2, CtE,Cprecat, Ccat, Ccatdead, H2uptake = init
        dEdt = -k1*CE*Cprecat - k2f*CE*CA1 + k2b*CtE*CA2
        dA1dt = k1*CE*Cprecat - k2f*CE*CA1 + k2b*CtE*CA2 + 2*k3*CtE*Cprecat
        dA2dt = k1*CE*Cprecat + k2f*CE*CA1 - k2b*CtE*CA2
        dtEdt = k2f*CE*CA1 - k2b*CtE*CA2 - k3*CtE*Cprecat
        dprecatdt = -k1*CE*Cprecat - k3*CtE*Cprecat + k4*Ccat*CH2 - k5*Cprecat
        dcatdt = k1*CE*Cprecat + k3*CtE*Cprecat - k4*Ccat*CH2
        dcatdeaddt = k5*Cprecat
        dH2uptakedt = k4*Ccat*CH2
        return dEdt, dA1dt, dA2dt, dtEdt, dprecatdt, dcatdt,dcatdeaddt, dH2uptakedt

    sol = integrate.solve_ivp(diff, [0,timespan[-1]], init, t_eval = timespan)
    

    return (np.average(sol.y[1]) + np.average(sol.y[2]))/(S0*Ccat0) #return what you want to optimize
    #i optimize for total alcohol concentration per substrate per catalyst

In [4]:
# give the bounds in pbounds of all optimized variables
pbounds = {'T': (273,453), 'P': (40,50), 'S0': (1,10), 'Ccat0' :(0.1,1)}

optimizer = BayesianOptimization(
    f=get_experiment,
    pbounds=pbounds,
)

In [5]:
#find the optimum, init points is the amount of random guesses, n_iter is the number of guesses, for kappa see the bayesianoptimization
#github, but 1.5 should be okay
optimizer.maximize(
    init_points=50,
    n_iter=500,
    kappa = 1.5,
)

|   iter    |  target   |   Ccat0   |     P     |    S0     |     T     |
-------------------------------------------------------------------------
|  1        |  0.7038   |  0.9359   |  41.05    |  1.273    |  444.7    |
|  2        |  0.3363   |  0.3088   |  46.89    |  3.893    |  322.1    |
|  3        |  0.2315   |  0.9375   |  43.31    |  7.864    |  352.6    |
|  4        |  0.2849   |  0.5661   |  48.58    |  5.93     |  347.4    |
|  5        |  0.281    |  0.3889   |  46.04    |  7.921    |  429.9    |
|  6        |  0.32     |  0.9551   |  40.22    |  3.832    |  309.6    |
|  7        |  0.2212   |  0.6547   |  45.28    |  9.359    |  380.6    |
|  8        |  0.4184   |  0.7658   |  42.21    |  4.271    |  417.6    |
|  9        |  0.2685   |  0.1961   |  42.77    |  6.67     |  366.7    |
|  10       |  0.2273   |  0.307    |  40.65    |  8.78     |  384.0    |
|  11       |  0.2032   |  0.6087   |  40.48    |  8.634    |  341.8    |
|  12       |  0.4008   |  0.3262   | 

|  67       |  0.7687   |  0.2713   |  48.97    |  1.012    |  435.8    |
|  68       |  0.7653   |  0.2216   |  48.77    |  1.02     |  433.3    |
|  69       |  0.7544   |  0.1      |  46.98    |  1.0      |  434.5    |
|  70       |  0.7798   |  0.4809   |  49.63    |  1.015    |  428.1    |
|  71       |  0.7557   |  0.7923   |  48.05    |  1.099    |  427.1    |
|  72       |  0.7626   |  0.1      |  50.0     |  1.0      |  425.5    |
|  73       |  0.753    |  1.0      |  50.0     |  1.0      |  427.3    |
|  74       |  0.7584   |  0.1      |  48.15    |  1.0      |  429.2    |
|  75       |  0.7677   |  0.3709   |  46.75    |  1.047    |  426.3    |
|  76       |  0.7451   |  0.1      |  40.0     |  1.0      |  424.7    |
|  77       |  0.7495   |  0.1      |  50.0     |  1.0      |  453.0    |
|  78       |  0.7731   |  0.4798   |  47.32    |  1.039    |  424.5    |
|  79       |  0.7531   |  0.1      |  44.14    |  1.0      |  423.4    |
|  80       |  0.7459   |  0.1312   | 

|  135      |  0.7626   |  0.1      |  48.65    |  1.0      |  401.7    |
|  136      |  0.7526   |  0.1      |  43.19    |  1.0      |  415.7    |
|  137      |  0.7508   |  1.0      |  49.17    |  1.0      |  429.3    |
|  138      |  0.1299   |  1.0      |  50.0     |  10.0     |  273.0    |
|  139      |  0.5171   |  0.1697   |  48.46    |  2.968    |  424.8    |
|  140      |  0.7537   |  0.1      |  43.64    |  1.0      |  412.9    |
|  141      |  0.7434   |  0.1      |  40.0     |  1.0      |  431.0    |
|  142      |  0.7592   |  0.824    |  43.08    |  1.007    |  393.8    |
|  143      |  0.7372   |  1.0      |  40.0     |  1.0      |  392.6    |
|  144      |  0.7491   |  1.0      |  45.51    |  1.0      |  396.5    |
|  145      |  0.7485   |  1.0      |  44.93    |  1.0      |  408.3    |
|  146      |  0.7487   |  0.1      |  41.03    |  1.0      |  413.7    |
|  147      |  0.7421   |  1.0      |  42.03    |  1.0      |  395.9    |
|  148      |  0.7477   |  0.1      | 

|  203      |  0.7496   |  0.1      |  41.79    |  1.0      |  401.1    |
|  204      |  0.7492   |  0.1      |  50.0     |  1.0      |  368.4    |
|  205      |  0.7602   |  0.1      |  50.0     |  1.0      |  432.3    |
|  206      |  0.7806   |  0.3587   |  49.0     |  1.02     |  388.9    |
|  207      |  0.754    |  1.0      |  49.23    |  1.0      |  388.3    |
|  208      |  0.4613   |  0.1759   |  49.31    |  3.332    |  387.3    |
|  209      |  0.7396   |  0.9767   |  40.38    |  1.002    |  419.3    |
|  210      |  0.755    |  0.8693   |  48.89    |  1.054    |  378.5    |
|  211      |  0.7804   |  0.4318   |  48.99    |  1.0      |  428.2    |
|  212      |  0.7535   |  0.1      |  50.0     |  1.0      |  373.6    |
|  213      |  0.7329   |  1.0      |  41.65    |  1.0      |  438.0    |
|  214      |  0.7389   |  0.1063   |  44.77    |  1.043    |  446.3    |
|  215      |  0.7507   |  1.0      |  48.09    |  1.0      |  384.9    |
|  216      |  0.7537   |  0.1      | 

|  271      |  0.7623   |  0.1      |  48.38    |  1.0      |  403.7    |
|  272      |  0.7503   |  0.4498   |  40.2     |  1.083    |  421.2    |
|  273      |  0.7653   |  0.1      |  50.0     |  1.0      |  410.6    |
|  274      |  0.7538   |  0.1      |  44.87    |  1.0      |  392.9    |
|  275      |  0.7414   |  1.0      |  50.0     |  1.0      |  448.2    |
|  276      |  0.7647   |  0.1      |  50.0     |  1.0      |  416.4    |
|  277      |  0.7337   |  0.1      |  40.0     |  1.0      |  453.0    |
|  278      |  0.7343   |  0.2062   |  40.5     |  1.054    |  450.2    |
|  279      |  0.7395   |  0.5807   |  40.16    |  1.072    |  447.0    |
|  280      |  0.7426   |  0.1      |  40.0     |  1.0      |  433.6    |
|  281      |  0.7327   |  0.1      |  41.3     |  1.0      |  368.3    |
|  282      |  0.7635   |  0.7313   |  42.52    |  1.008    |  391.3    |
|  283      |  0.7612   |  0.1      |  48.53    |  1.0      |  395.2    |
|  284      |  0.7523   |  0.1741   | 

|  339      |  0.7306   |  0.1      |  50.0     |  1.0      |  351.8    |
|  340      |  0.7126   |  0.9918   |  46.72    |  1.01     |  347.0    |
|  341      |  0.7325   |  1.0      |  50.0     |  1.0      |  357.4    |
|  342      |  0.7048   |  1.0      |  43.12    |  1.0      |  345.5    |
|  343      |  0.7405   |  0.2918   |  46.47    |  1.061    |  358.3    |
|  344      |  0.7298   |  0.1444   |  43.29    |  1.023    |  360.1    |
|  345      |  0.7379   |  0.8493   |  46.45    |  1.021    |  356.7    |
|  346      |  0.7436   |  0.5577   |  42.77    |  1.031    |  356.9    |
|  347      |  0.7136   |  1.0      |  50.0     |  1.0      |  344.1    |
|  348      |  0.724    |  0.1      |  50.0     |  1.0      |  347.4    |
|  349      |  0.7305   |  1.0      |  46.11    |  1.0      |  361.7    |
|  350      |  0.7233   |  0.7955   |  49.86    |  1.014    |  338.9    |
|  351      |  0.727    |  0.4678   |  46.29    |  1.028    |  339.4    |
|  352      |  0.6677   |  0.1869   | 

|  407      |  0.7588   |  0.1      |  47.54    |  1.0      |  392.9    |
|  408      |  0.7769   |  0.7444   |  47.77    |  1.0      |  397.1    |
|  409      |  0.7396   |  1.0      |  41.31    |  1.0      |  391.2    |
|  410      |  0.7784   |  0.4645   |  47.77    |  1.04     |  397.7    |
|  411      |  0.7887   |  0.4688   |  49.25    |  1.0      |  406.7    |
|  412      |  0.7791   |  0.5914   |  48.91    |  1.0      |  428.1    |
|  413      |  0.7522   |  1.0      |  49.4     |  1.0      |  383.0    |
|  414      |  0.7734   |  0.7784   |  48.16    |  1.015    |  401.3    |
|  415      |  0.7393   |  0.1      |  50.0     |  1.0      |  358.7    |
|  416      |  0.7554   |  0.1      |  44.57    |  1.0      |  405.2    |
|  417      |  0.7767   |  0.2499   |  48.19    |  1.005    |  399.3    |
|  418      |  0.7455   |  0.1      |  40.0     |  1.0      |  422.7    |
|  419      |  0.7539   |  0.1      |  47.26    |  1.0      |  436.9    |
|  420      |  0.7368   |  0.7267   | 

|  475      |  0.788    |  0.4309   |  49.18    |  1.0      |  392.5    |
|  476      |  0.7859   |  0.472    |  48.37    |  1.0      |  388.9    |
|  477      |  0.7844   |  0.683    |  49.63    |  1.0      |  405.7    |
|  478      |  0.7815   |  0.4753   |  46.03    |  1.0      |  406.4    |
|  479      |  0.7808   |  0.7106   |  48.87    |  1.0      |  392.1    |
|  480      |  0.7796   |  0.6778   |  47.17    |  1.0      |  401.9    |
|  481      |  0.7732   |  0.4525   |  49.03    |  1.0      |  440.5    |
|  482      |  0.779    |  0.248    |  49.29    |  1.0      |  392.8    |
|  483      |  0.7696   |  0.8198   |  46.87    |  1.0      |  402.4    |
|  484      |  0.7907   |  0.5249   |  50.0     |  1.0      |  398.9    |
|  485      |  0.7553   |  0.1      |  45.06    |  1.0      |  420.9    |
|  486      |  0.4825   |  0.1      |  40.0     |  3.105    |  424.7    |
|  487      |  0.7575   |  0.1      |  46.51    |  1.0      |  395.0    |
|  488      |  0.2242   |  0.1      | 

|  543      |  0.7517   |  0.1      |  42.6     |  1.0      |  413.9    |
|  544      |  0.7651   |  0.5397   |  43.99    |  1.0      |  436.6    |
|  545      |  0.7549   |  0.1      |  46.85    |  1.0      |  432.7    |
|  546      |  0.7793   |  0.4694   |  47.32    |  1.0      |  423.9    |
|  547      |  0.7464   |  1.0      |  45.31    |  1.0      |  387.3    |
|  548      |  0.7441   |  0.1      |  40.61    |  1.0      |  389.1    |
|  549      |  0.74     |  0.1722   |  42.05    |  1.096    |  412.5    |
|  550      |  0.7543   |  0.1      |  46.23    |  1.0      |  387.3    |


In [7]:
#Since the data we use is synthesized and not experimental
#Now we can validate how accurate the guess is with the system we initially put into chemfit
def get_experiment(T,P,S0,Ccat0): #the function takes the to optimize variables
    'This function is to illustrate the bayesian optimizer'
    R= 8.314 #define needed constants
    KH= 1282
    CH2 = P/KH
    k1 = 4*np.exp(-17507/(R*T)) #now if i have found my arrhenius constants and activation energies i put them in like this
    k2f = 1.6*np.exp(-14058/(R*T))
    k2b = 1*np.exp(-16473/(R*T))
    k3 = 5*np.exp(-12132/(R*T))
    k4 = 2*np.exp(-12396/(R*T))
    k5 = 40*np.exp(-24312/(R*T))
    init = [S0,0,0,0,Ccat0,0,0,0] #define how the initial conditon is made from the optimized variables
    def diff(x, init): #the differential system to simulate
        CE, CA1, CA2, CtE,Cprecat, Ccat, Ccatdead, H2uptake = init
        dEdt = -k1*CE*Cprecat - k2f*CE*CA1 + k2b*CtE*CA2
        dA1dt = k1*CE*Cprecat - k2f*CE*CA1 + k2b*CtE*CA2 + 2*k3*CtE*Cprecat
        dA2dt = k1*CE*Cprecat + k2f*CE*CA1 - k2b*CtE*CA2
        dtEdt = k2f*CE*CA1 - k2b*CtE*CA2 - k3*CtE*Cprecat
        dprecatdt = -k1*CE*Cprecat - k3*CtE*Cprecat + k4*Ccat*CH2 - k5*Cprecat
        dcatdt = k1*CE*Cprecat + k3*CtE*Cprecat - k4*Ccat*CH2
        dcatdeaddt = k5*Cprecat
        dH2uptakedt = k4*Ccat*CH2
        return dEdt, dA1dt, dA2dt, dtEdt, dprecatdt, dcatdt,dcatdeaddt, dH2uptakedt

    sol = integrate.solve_ivp(diff, [0,timespan[-1]], init, t_eval = timespan)
    

    return (np.average(sol.y[1]) + np.average(sol.y[2]))/(S0*Ccat0) #return what you want to optimize
    #i optimize for total alcohol concentration per substrate per catalyst

In [8]:
# put in the parameters we found to see if the yield is accurate for the system with the accurate parameters
print(get_experiment(398.9,50,0.5249,1))

0.7912233673152701
